In [ ]:
%load_ext autoreload
%autoreload 2
# default_exp plugin.pluginbase

In [ ]:
# export
from pymemri.data.schema import *
from pymemri.pod.client import PodClient, DEFAULT_POD_ADDRESS
from pymemri.imports import *
from os import environ
from abc import ABCMeta
import abc
import json
import importlib
import string
from enum import Enum
from pymemri.pod.client import PodClient
from fastscript import *
import os

In [ ]:
# hide
from nbdev.showdoc import *

# Plugins

PluginBase is the plugin class that the simplest plugin inherits.

Inheriting class should implement:

- run()           that implements the logic of the plugin
- add_to_schema() for plugin specific item types

In [ ]:
# export
POD_FULL_ADDRESS_ENV        = 'POD_FULL_ADDRESS'
POD_TARGET_ITEM_ENV         = 'POD_TARGET_ITEM'
POD_OWNER_KEY_ENV           = 'POD_OWNER'
POD_AUTH_JSON_ENV           = 'POD_AUTH_JSON'

In [ ]:
# export
# hide
class PluginBase(Item, metaclass=ABCMeta):
    """Base class for plugins"""
    properties = Item.properties + ["name", "repository", "icon", "data_query", "bundleImage",
                                    "runDestination", "pluginClass"]
    edges = Item.edges
    
    def __init__(self, name=None, repository=None, icon=None, query=None, bundleImage=None, runDestination=None,
                 pluginClass=None, **kwargs):
        if pluginClass is None: pluginClass=self.__class__.__name__
        super().__init__(**kwargs)
        self.name = name
        self.repository = repository
        self.icon = icon
        self.query = query
        self.bundleImage = bundleImage
        self.runDestination = runDestination
        self.pluginClass = pluginClass
    
    @abc.abstractmethod
    def run(self):
        raise NotImplementedError()
        
    @abc.abstractmethod    
    def add_to_schema(self, client):
        raise NotImplementedError()

In [ ]:
# export
# hide
class PluginRun(Item):
    properties = Item.properties + ["targetItemId", "pluginModule", "pluginName", "config", "containerImage",
                                    "state"]
    edges = PluginBase.edges + ["view"]

    def __init__(self, containerImage, pluginModule, pluginName, config="", state=None, view=None, targetItemId=None,
                 **kwargs):
        """
                PluginRun defines a the run of plugin `plugin_module.plugin_name`,
        with an optional `config` string.

        Args:
            plugin_module (str): module of the plugin.
            plugin_name (str): class name of the plugin.
            config (str, optional): Optional plugin configuration. For example, 
                this could be a `json.dumps` of a configuration dict. Defaults to None.
        """
        super().__init__(**kwargs)
        self.pluginModule = pluginModule
        self.pluginName = pluginName
        self.config = config
        self.containerImage=containerImage
        id_ = "".join([random.choice(string.hexdigits) for i in range(32)]) if targetItemId is None else targetItemId
        self.targetItemId=id_
        self.id=id_
        self.state=state

        self.view = view if view is not None else []

## Creating a plugin

The memri [pod](https://gitlab.memri.io/memri/pod) uses a plugin system to add features to the backend memri backend. Plugins can import your data (importers), change your data (indexers), or call other serivces. Users can define their own plugins to add new behaviour to their memri app. Let's use the following plugin as an example of how we can start plugins.

In [ ]:
# export
# hide
class MyItem(Item):
    properties = Item.properties + ["name", "age"]
    edges = Item.edges
    def __init__(self, name=None, age=None, **kwargs):
        super().__init__(**kwargs)
        self.name = name
        self.age = age

class MyPlugin(PluginBase):
    """"""
    properties = PluginBase.properties + ["containerImage"]
    edges= PluginBase.edges
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
    def run(self, run, client):
        print("running")
        client.create(MyItem("some person", 20))
        
    def add_to_schema(self, client):
        client.add_to_schema(MyItem("my name", 10)) 

In [ ]:
# hide
MyPlugin()

MyPlugin (#None)

```python
class MyItem(Item):
    properties = Item.properties + ["name", "age"]
    edges = Item.edges
    def __init__(self, name=None, age=None, **kwargs):
        super().__init__(**kwargs)
        self.name = name
        self.age = age

class MyPlugin(PluginBase):
    """"""
    properties = PluginBase.properties
    edges= PluginBase.edges
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
    def run(self, run, client):
        print("running")
        client.create(MyItem("some person", 20))
        
    def add_to_schema(self, client):
        client.add_to_schema(MyItem("my name", 10)) 
```

Memri plugins need to define at least 2 methods: `.run()` and `.add_to_schema()`. `.run()` defines the logic of the plugin. `.add_to_schema()` defines the schema for the plugin in the pod. Note that currently, `add_to_schema` requires all item to **have all properties defined that are used in the plugin**. In the future, we might replace add_to_schema, to be done automatically, based on a declarative schema defined in the plugin. 

## Helper classes -

In [ ]:
# hide
# export
def get_plugin(plugin_module, plugin_name):
    try:
        module = importlib.import_module(plugin_module)
        plugin_cls = getattr(module, plugin_name)
        return plugin_cls()
    except (ImportError, AttributeError):
        raise ImportError(f"Unknown plugin: {plugin_module}.{plugin_name}")

def run_plugin_from_run_id(run_id, client, return_plugin=False):
    """
    Args:
        run_id (int): id of the PluginRun
        client (PodClient): client containing PluginRun
        return_plugin (bool): Returns created plugin instance for testing purposes.
    """
    run = client.get(run_id)
    plugin = get_plugin(run.pluginModule, run.pluginName)
    plugin.add_to_schema(client)
    plugin.run(run, client)

    return plugin if return_plugin else None

In [ ]:
# export
# hide
def register_base_schemas(client):
    try:
        assert client.add_to_schema(PluginRun("", "", "", "", ""))
        assert client.add_to_schema(CVUStoredDefinition(name="", definition=""))
    except Exception as e:
        raise ValueError("Could not add base schema")

In [ ]:
# hide
# skip
client = PodClient()
run = PluginRun("pymemri", "pymemri.plugin.pluginbase", "MyPlugin", "my_config_string", state="not started")
assert client.add_to_schema(PluginRun("", "", "", "", ""))
assert client.create(run)
run = client.get(run.id)
run_plugin_from_run_id(run.id, client)

running


In [ ]:
# export
def _run_plugin(client, plugin_run_id=None, verbose=False):
    """Runs an plugin, you can either provide the run settings as parameters to this function (for local testing)
    or via environment variables (this is how the pod communicates with plugins)."""
    register_base_schemas(client)
    run_plugin_from_run_id(plugin_run_id, client)

In [ ]:
# hide
# skip
_run_plugin(client=client, plugin_run_id=run.id)

running


In [ ]:
# export
# hide
def _parse_env(env):
    try:
        pod_full_address = env.get(POD_FULL_ADDRESS_ENV, DEFAULT_POD_ADDRESS)
        plugin_run_json  = json.loads(str(env[POD_TARGET_ITEM_ENV]))
        print(plugin_run_json)
        plugin_run_id    = plugin_run_json["id"]
        owner_key        = env.get(POD_OWNER_KEY_ENV)
        pod_auth_json    = json.loads(str(env.get(POD_AUTH_JSON_ENV)))
#         database_key = pod_service_payload[DATABASE_KEY_ENV]
#         owner_key    = pod_service_payload[OWNER_KEY_ENV]
        return pod_full_address, plugin_run_id, pod_auth_json, owner_key
    except KeyError as e:
        raise Exception('Missing parameter: {}'.format(e)) from None


## Running your plugin using the CLI

Plugins can be started using the pymemri `run_plugin` or `run_plugin_from_pod` CLI. With `run_plugin` the plugin is invoked directly by spawning a new python process, while `run_plugin_from_pod` requests the pod to spawn a new process, docker container, or kubernetes container, which in calls `run_plugin` (for more info see `run_plugin_from_pod`. When using `run_plugin`, you can either pass your run arguments as parameters, or set them as environment variables. If both are set, the CLI will use the passed arguments.

In [ ]:
# export
@call_parse
def run_plugin(pod_full_address:Param("The pod full address", str)=None,
               plugin_run_id:Param("Run id of the plugin to be executed", str)=None,
               database_key:Param("Database key of the pod", str)=None,
               owner_key:Param("Owner key of the pod", str)=None,
               container:Param("Pod container to run frod", str)=None):

    env = os.environ
    params = [pod_full_address, plugin_run_id, database_key, owner_key]
    
    if all([p is None for p in params]):
        print("Reading `run_plugin()` parameters from environment variables")
        pod_full_address, plugin_run_id, pod_auth_json, owner_key = _parse_env(env)
        database_key=None
    else:
        print("Used arguments passed to `run_plugin()` (ignoring environment)")
        pod_auth_json=None
        if (None in params):
            raise ValueError(f"Defined some params to run indexer, but not all. Missing \
                             {[p for p in params if p is None]}")
    client = PodClient(url=pod_full_address, database_key=database_key, owner_key=owner_key,
                       auth_json=pod_auth_json)
    for name, val in [("pod_full_address", pod_full_address), ("plugin_run_id", plugin_run_id),
                      ("owner_key", owner_key), ("auth_json", pod_auth_json)]:
        print(f"{name}={val}")
    print()
    _run_plugin(client=client, plugin_run_id=plugin_run_id)

To start a plugin on your local machine, you can use the CLI. This will create a client for you, and run the code defined in `<myplugin>.run()`

In [ ]:
# skip
client = PodClient()
run = PluginRun(containerImage="pymemri", pluginModule="pymemri.plugin.pluginbase",
                pluginName="MyPlugin", config="", state="not started")
assert client.add_to_schema(PluginRun("", "", "", "", "")) and client.create(run)

In [ ]:
!run_plugin --pod_full_address=$DEFAULT_POD_ADDRESS --owner_key=$client.owner_key \
            --database_key=$client.database_key --container="pymemri" --plugin_run_id=$run.id

Used arguments passed to `run_plugin()` (ignoring environment)
pod_full_address=http://localhost:3030
plugin_run_id=1bf06Bfb7B63f114FcfB8DBC67eA49Be
owner_key=3961574559904730128514448689870083724159258570216156665995117538
auth_json=None

running


> Note: The data that is created here should be in the pod in order for this to work

## Run from pod 

In production, we start plugins by making an API call to the pod, which in turn creates an environment for the plugin and starts it using docker containers, kubernetes containers or a shell script. We can start this process using the `run_plugin_from_pod` CLI. **Note that when using docker, provided container name should be "installed" within the Pod environemnt (e.g. `docker build -t pymemri .` for this repo) in order to start it.** 

![running a plugin](images/running_a_plugin.svg)

In [ ]:
# export
from fastcore.script import call_parse, Param
import os

@call_parse
def run_plugin_from_pod(pod_full_address:Param("The pod full address", str)=None,
                        database_key:Param("Database key of the pod", str)=None,
                        owner_key:Param("Owner key of the pod", str)=None,
                        container:Param("Pod container to run frod", str)=None,
                        plugin_module:Param("Plugin module", str)=None,
                        plugin_name:Param("Plugin class name", str)=None,
                        settings_file:Param("Plugin settings (json)", str)=None):
    params = [pod_full_address, database_key, owner_key, container, plugin_module, plugin_name]
    if (None in params):
        raise ValueError(f"Defined some params to run indexer, but not all. Missing \
                         {[p for p in params if p is None]}")
    client = PodClient(url=pod_full_address, database_key=database_key, owner_key=owner_key)
    for name, val in [("pod_full_address", pod_full_address), ("owner_key", owner_key)]:
        print(f"{name}={val}")

    if settings_file is not None:
        with open(settings_file, 'r') as f:
            settings = f.read()
    else:
        settings = None
    
    register_base_schemas(client)
    run = PluginRun(container, plugin_module, plugin_name, settings)
    print(f"\ncalling the `create` api on {pod_full_address} to make your Pod start " 
          f"a plugin with id {run.id}.")
    print(f"*Check the pod log/console for debug output.*")
    client.create(run)
    print(f"Created PluginRun: {run.id}")
    return run


In [ ]:
client = PodClient()

In [ ]:
!run_plugin_from_pod --pod_full_address=$DEFAULT_POD_ADDRESS --owner_key=$client.owner_key \
                     --database_key=$client.database_key --container="pymemri" \
                     --plugin_module="pymemri.plugin.pluginbase" --plugin_name="MyPlugin"

pod_full_address=http://localhost:3030
owner_key=4566065835753341432440539540364977361089552669232085362436171995

calling the `create` api on http://localhost:3030 to make your Pod start a plugin with id cc01FaEbAf5DC88ADc27e408a790b8D6.
*Check the pod log/console for debug output.*
Created PluginRun: cc01FaEbAf5DC88ADc27e408a790b8D6


## Appendix -

In [ ]:
# hide
# client.start_plugin("pymemri", run.id)

In [ ]:
# hide
# class StartPlugin(Item):
#     properties = Item.properties + ["container", "targetItemId"]
#     edges = Item.edges
#     def __init__(self, container=None, targetItemId=None, **kwargs):
#         super().__init__(**kwargs)
#         self.container = container
#         self.targetItemId = targetItemId

In [ ]:
# hide
# class PluginSettings(Item):
#     def __init__(self, settings_dict):
#         self.settings_dict=settings_dict
        
#     def __getattr__(self, k):
#         if k in self.settings_dict:
#             return self.settings_dict[k]
        
#     @classmethod
#     def from_string(cls, s):
#         objs = json.loads(s)
#         cls(objs)

In [ ]:
# hide
# # export
# def generate_test_env(client, indexer_run):
#     payload = json.dumps({DATABASE_KEY_ENV: client.database_key, OWNER_KEY_ENV: client.owner_key})
              
#     return {POD_FULL_ADDRESS_ENV: DEFAULT_POD_ADDRESS,
#             POD_TARGET_ITEM: indexer_run.id,
#             POD_SERVICE_PAYLOAD_ENV: payload}

In [ ]:
# hide
# run_plugin(env=generate_test_env(client, run))

# Export -

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted basic.ipynb.
Converted data.photo.ipynb.
Converted importers.Importer.ipynb.
Converted importers.util.ipynb.
Converted index.ipynb.
Converted indexers.indexer.ipynb.
Converted itembase.ipynb.
Converted plugin.pluginbase.ipynb.
Converted pod.client.ipynb.
